# Logging Concurrently

Once we've got the hang of using `rubicon`, we can expand on our project from the *Iris Classifier* example. 
Let's see how a few other popular `scikit-learn` models perform with the Iris dataset. `rubicon` logging is totally 
thread-safe, so we can test a lot of model configurations at once.

In [1]:
from rubicon import Rubicon

root_dir = "./rubicon-root"

rubicon = Rubicon(persistence="filesystem", root_dir=root_dir)
project = rubicon.create_project("Concurrent Experiments", description="Training multiple models in parallel")

print(project)

Project(name='Concurrent Experiments', id='4e8ba1ae-ecf8-4976-9650-9ba3701a4fc3', description='Training multiple models in parallel', github_url=None, training_metadata=None, created_at=datetime.datetime(2021, 4, 5, 18, 15, 16, 189837))


For a recap of the contents of the Iris dataset, check out `iris.DESCR` and `iris.data`. We'll put together
a training dataset using a subset of the data.

In [2]:
from datetime import datetime
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

random_state = int(datetime.utcnow().timestamp())

iris = load_iris()
iris_datasets = train_test_split(iris['data'], iris['target'], random_state=random_state)

We'll use `run_experiment` to log a new experiment to the provided `project` then train, run and log a model of type
`classifier_cls` using the training and testing data in `iris_datasets`.

In [3]:
import pandas as pd
from collections import namedtuple

SklearnTrainingMetadata = namedtuple("SklearnTrainingMetadata", "module_name method")

def run_experiment(project, classifier_cls, iris_datasets, **kwargs):
    X_train, X_test, y_train, y_test = iris_datasets
    
    experiment = project.log_experiment(
        training_metadata=[SklearnTrainingMetadata("sklearn.datasets", "load_iris")],
        model_name=classifier_cls.__name__,
        tags=[classifier_cls.__name__],
    )
    
    for key, value in kwargs.items():
        experiment.log_parameter(key, value)
    
    n_features = len(iris.feature_names)
    experiment.log_parameter("n_features", n_features)
    
    for name in iris.feature_names:
        experiment.log_feature(name)
        
    classifier = classifier_cls(**kwargs)
    classifier.fit(X_train, y_train)
    classifier.predict(X_test)
    
    accuracy = classifier.score(X_test, y_test)
    
    experiment.log_metric("accuracy", accuracy)

    if accuracy >= .95:
        experiment.add_tags(["success"])
    else:
        experiment.add_tags(["failure"])

This time we'll take a look at two more classifiers, **decision tree** and **k-neighbors**, in addition to the **random forest** classifier we used in the last example. Each classifier will be ran across four sets of parameters (provided as `kwargs` to `run_experiment`), for a total of 12 experiments. Here, we'll build up a list of processes that will run each experiment in parallel.

In [4]:
import multiprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

processes = []

for n_estimators in [10, 20, 30, 40]:
    processes.append(multiprocessing.Process(
        target=run_experiment, args=[project, RandomForestClassifier, iris_datasets],
        kwargs={"n_estimators": n_estimators, "random_state": random_state},
    ))
    
for criterion in ["gini", "entropy"]:
    for splitter in ["best", "random"]:
        processes.append(multiprocessing.Process(
            target=run_experiment, args=[project, DecisionTreeClassifier, iris_datasets],
            kwargs={"criterion": criterion, "splitter": splitter, "random_state": random_state},
        ))

for n_neighbors in [5, 10, 15, 20]:
    processes.append(multiprocessing.Process(
        target=run_experiment, args=[project, KNeighborsClassifier, iris_datasets],
        kwargs={"n_neighbors": n_neighbors},
    ))

Let's run all our experiments in parallel!

In [5]:
for process in processes:
    process.start()
    
for process in processes:
    process.join()

Now we can validate that we successfully logged all 12 experiments to our project.

In [6]:
len(project.experiments())

12

Let's see which experiments we tagged as successful and what type of model they used.

In [7]:
for e in project.experiments(tags=["success"]):    
    print(f"experiment {e.id} was successful using a {e.model_name}")

experiment 05424cbd-beb4-4171-b75b-bfda5c6ba729 was successful using a RandomForestClassifier
experiment 1cd670e8-745a-40de-89db-66d7704fb885 was successful using a RandomForestClassifier
experiment 2bdb17d3-3c08-4245-b918-a0267db14c34 was successful using a RandomForestClassifier
experiment 3ac29a9f-6043-4239-bc36-5cba4b64be11 was successful using a KNeighborsClassifier
experiment 7334f1e3-129e-4681-a23e-0f41ae810535 was successful using a DecisionTreeClassifier
experiment a12d8518-9d53-4b80-84ab-22be1f8b24e8 was successful using a RandomForestClassifier
experiment b1940343-6c58-41f2-939b-05c2448ee94f was successful using a DecisionTreeClassifier
experiment cee36678-90b2-46e1-b6be-e52b88871f1c was successful using a KNeighborsClassifier
experiment ed3cba7a-da3d-4184-991d-58c03222d290 was successful using a KNeighborsClassifier
experiment f15b95c9-4dfa-4b9a-9c89-8d703c9f2f0e was successful using a KNeighborsClassifier


We can also take a deeper look at any of our experiments.

In [8]:
experiment = project.experiments()[0]

print(f"training_metadata: {SklearnTrainingMetadata(*experiment.training_metadata)}")
print(f"tags: {experiment.tags}")
print("parameters:")
for parameter in experiment.parameters():
    print(f"\t{parameter.name}: {parameter.value}")
print("features:")
for feature in experiment.features():
    print(f"\t{feature.name}")
print("metrics:")
for metric in experiment.metrics():
    print(f"\t{metric.name}: {metric.value}")

training_metadata: SklearnTrainingMetadata(module_name='sklearn.datasets', method='load_iris')
tags: ['RandomForestClassifier', 'success']
parameters:
	n_estimators: 20
	n_features: 4
	random_state: 1617646516
features:
	petal length (cm)
	petal width (cm)
	sepal length (cm)
	sepal width (cm)
metrics:
	accuracy: 0.9736842105263158


Model developers can take advantage of `rubicon`'s thread-safety to test tons of models at once and collect results
in a standardized format to analyze which ones performed the best. `rubicon` can even be run in more advanced
distributed setups, like on a *Dask* cluster.